In [1]:
import pyarrow.parquet as pq
import pandas as pd
import requests
import io

In [2]:
# Load data
url = 'https://storage.googleapis.com/nyc-taxi-analysis-bucket-gv/fhvhv_tripdata_2023-01.parquet'
response = requests.get(url)

hv_trips = pq.read_table(io.BytesIO(response.content)).to_pandas()
hv_trips = hv_trips.drop_duplicates().reset_index(drop=True)
hv_trips['trip_id'] = hv_trips.index

In [4]:
lookup_url = 'https://storage.googleapis.com/nyc-taxi-analysis-bucket-gv/taxi_zone_lookup.csv'
response = requests.get(lookup_url)
location_dim = pd.read_csv(io.StringIO(response.text), sep=',')

In [5]:
license_data = {
    'hvfhs_license_num': ['HV0002', 'HV0003', 'HV0004', 'HV0005'],
    'license_name': ['juno', 'uber', 'via', 'lyft']
}
license_dim = pd.DataFrame(license_data)

In [6]:
unique_datetimes = pd.Series(pd.unique(hv_trips[['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']].values.ravel('K')))
unique_datetimes = unique_datetimes.dropna()

In [7]:
datetime_dim = pd.DataFrame(data=unique_datetimes, columns=['datetime'])
datetime_dim['year'] = datetime_dim['datetime'].dt.year
datetime_dim['month'] = datetime_dim['datetime'].dt.month
datetime_dim['day'] = datetime_dim['datetime'].dt.day
datetime_dim['hour'] = datetime_dim['datetime'].dt.hour
datetime_dim['minute'] = datetime_dim['datetime'].dt.minute
datetime_dim['day_of_week'] = datetime_dim['datetime'].dt.day_name()
datetime_dim['is_weekend'] = datetime_dim['datetime'].dt.weekday >= 5

In [9]:
fact_table = hv_trips[[
'trip_id', 
'hvfhs_license_num', 
'request_datetime', 
'on_scene_datetime', 
'pickup_datetime', 
'dropoff_datetime', 
'PULocationID',
'DOLocationID',
'trip_miles',
'trip_time',
'base_passenger_fare',
'tolls',
'bcf',
'sales_tax',
'congestion_surcharge',
'airport_fee',
'tips',
'driver_pay',
'wav_request_flag',
'wav_match_flag',
]]

In [10]:
fact_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18479031 entries, 0 to 18479030
Data columns (total 20 columns):
 #   Column                Dtype         
---  ------                -----         
 0   trip_id               int64         
 1   hvfhs_license_num     object        
 2   request_datetime      datetime64[us]
 3   on_scene_datetime     datetime64[us]
 4   pickup_datetime       datetime64[us]
 5   dropoff_datetime      datetime64[us]
 6   PULocationID          int64         
 7   DOLocationID          int64         
 8   trip_miles            float64       
 9   trip_time             int64         
 10  base_passenger_fare   float64       
 11  tolls                 float64       
 12  bcf                   float64       
 13  sales_tax             float64       
 14  congestion_surcharge  float64       
 15  airport_fee           float64       
 16  tips                  float64       
 17  driver_pay            float64       
 18  wav_request_flag      object        
 19

In [11]:
fact_table.head()

,trip_id,hvfhs_license_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,wav_request_flag,wav_match_flag
0,0,HV0003,2023-01-01 00:18:06,2023-01-01 00:19:24,2023-01-01 00:19:38,2023-01-01 00:48:07,48,68,0.94,1709,25.95,0.0,0.78,2.30,2.75,0.0,5.22,27.83,N,N
1,1,HV0003,2023-01-01 00:48:42,2023-01-01 00:56:20,2023-01-01 00:58:39,2023-01-01 01:33:08,246,163,2.78,2069,60.14,0.0,1.80,5.34,2.75,0.0,0.00,50.15,N,N
2,2,HV0003,2023-01-01 00:15:35,2023-01-01 00:20:14,2023-01-01 00:20:27,2023-01-01 00:37:54,9,129,8.81,1047,24.37,0.0,0.73,2.16,0.00,0.0,0.00,20.22,N,N
3,3,HV0003,2023-01-01 00:35:24,2023-01-01 00:39:30,2023-01-01 00:41:05,2023-01-01 00:48:16,129,129,0.67,431,13.80,0.0,0.41,1.22,0.00,0.0,0.00,7.90,N,N
4,4,HV0003,2023-01-01 00:43:15,2023-01-01 00:51:10,2023-01-01 00:52:47,2023-01-01 01:04:51,129,92,4.38,724,20.49,0.0,0.61,1.82,0.00,0.0,0.00,16.48,N,N


In [13]:
location_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       264 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [14]:
location_dim.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [15]:
license_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   hvfhs_license_num  4 non-null      object
 1   license_name       4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [16]:
license_dim.head()

,hvfhs_license_num,license_name
0,HV0002,juno
1,HV0003,uber
2,HV0004,via
3,HV0005,lyft


In [17]:
datetime_dim.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2679266 entries, 0 to 2679266
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   datetime     datetime64[us]
 1   year         int32         
 2   month        int32         
 3   day          int32         
 4   hour         int32         
 5   minute       int32         
 6   day_of_week  object        
 7   is_weekend   bool          
dtypes: bool(1), datetime64[us](1), int32(5), object(1)
memory usage: 115.0+ MB


In [18]:
datetime_dim.head()

,datetime,year,month,day,hour,minute,day_of_week,is_weekend
0,2023-01-01 00:18:06,2023,1,1,0,18,Sunday,True
1,2023-01-01 00:48:42,2023,1,1,0,48,Sunday,True
2,2023-01-01 00:15:35,2023,1,1,0,15,Sunday,True
3,2023-01-01 00:35:24,2023,1,1,0,35,Sunday,True
4,2023-01-01 00:43:15,2023,1,1,0,43,Sunday,True
